In [12]:
#pip install -q pyomo
#pip install gurobipy

import pyomo.environ as pyo

# sets (all caps)
TRANSPORTATION = ['Helicopter', 'Plane']

# parameters ((descriptive names in lowercase letters))

cost = {'Helicopter': 5000, 'Plane': 20000}

supplies = {'Helicopter': 17, 'Plane': 200}

medical_staff = {'Helicopter': 20, 'Plane': 10}

available = {'Helicopter': 10, 'Plane': 6}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(TRANSPORTATION, domain=pyo.NonNegativeIntegers)


#Objective
def obj_rule(model):
    return sum(model.x[i] * cost[i]for i in TRANSPORTATION)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def supplies_c(model):
    return (sum(supplies[i]*model.x[i] for i in TRANSPORTATION)>= 1000)

model.suppies_const = pyo.Constraint(rule=supplies_c)

def medical(model):
    return (sum(medical_staff[i]*model.x[i] for i in TRANSPORTATION) >= 200)

model.medical_const = pyo.Constraint(rule=medical)

def availability(model, i):
    return model.x[i] <= available[i]

model.availability_const = pyo.Constraint(TRANSPORTATION, rule=availability)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Cost: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in TRANSPORTATION:
        print(f"{i}  = {pyo.value(model.x[i])}")
        

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmpx4a5a57_.pyomo.lp
Reading time = 0.00 seconds
x1: 4 rows, 2 columns, 6 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xf7700062
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 140000 

Optimal solution